# Iceflow solver

### Set up Julia environment

In [ ]:
# activate the environment
using Pkg
Pkg.activate("./")
Pkg.instantiate()
# activate packages to use
using Printf, CairoMakie

### Create data: generate the bedrock elevation `B` and the equilibrium line altitude `ELA` data for a given grid resolutions `nx, ny`.

In [ ]:
# constants
const s2yr = 31557600 # seconds to years
# physics
lx, ly = 250000.0, 200000.0  # domain size [m]
β      = 0.01                # mass-balance slope (data)
c      = 2.0                 # mass-balance limiter
ρg     = 910.0 * 9.81        # ice density x gravity
a1     = 1.9e-24 * ρg^3 * s2yr
a2     = 5.7e-20 * ρg^3 * s2yr
B0     = 3500.0              # mean height [m]
# numerics
nx, ny = 256, 256            # grid resolution
dx, dy = lx / nx, ly / ny    # grid spacing
nt     = 1e4                 # number of time steps
dt     = 0.1                 # time step [yr]
nout   = 1e3                 # visu and error checking interval
ϵ      = 1e-4                # steady state tolerance
# initial conditions (data)
xc     = LinRange(-lx / 2 + dx / 2, lx / 2 - dx / 2, nx)
yc     = LinRange(-ly / 2 + dy / 2, ly / 2 - dy / 2, ny)
Xc, Yc = [x for x = xc, y = yc], [y for x = xc, y = yc]
B      = zeros(nx, ny)
ELA    = zeros(nx, ny)
@. B   = B0 * exp(-Xc^2 / 1e10 - Yc^2 / 1e9) + B0 * exp(-Xc^2 / 1e9 - (Yc - ly / 8) * (Yc - ly / 8) / 1e10)
@. ELA = 2150 + 900 * atan(Yc / ly)

### Pre-processing and array initialisation

In [ ]:
# initialise
S      = zeros(nx  , ny  )
dSdx   = zeros(nx-1, ny  )
dSdy   = zeros(nx  , ny-1)
Snorm  = zeros(nx-1, ny-1)
D      = zeros(nx-1, ny-1)
qx     = zeros(nx-1, ny-2)
qy     = zeros(nx-2, ny-1)
H      = zeros(nx  , ny  )
M      = zeros(nx  , ny  )
H0     = zeros(nx  , ny  )

### Helper function definition

In [ ]:
# averaging
@views av(A) = 0.25 .* (A[1:end-1, 1:end-1] .+ A[1:end-1, 2:end] .+ A[2:end, 1:end-1] .+ A[2:end, 2:end])
@views avx(A) = 0.5 .* (A[1:end-1, :] .+ A[2:end, :])
@views avy(A) = 0.5 .* (A[:, 1:end-1] .+ A[:, 2:end])
# Compute the effective nonlinear diffusion coefficient `D` for SIA model.
@views function compute_D!(D, H, S, dSdx, dSdy, Snorm, a1, a2, dx, dy)
    dSdx  .= diff(S, dims=1) ./ dx
    dSdy  .= diff(S, dims=2) ./ dy
    Snorm .= ((avy(dSdx) .^ 2) .+ (avx(dSdy) .^ 2)) .^ 0.5
    D     .= ((a1 .* av(H) .^ 5) .+ (a2 .* av(H) .^ 3)) .* Snorm .^ 2
    return
end

### Time loop - action

In [ ]:
# time loop
for it = 1:nt
    H0 .= H
    S  .= B .+ H
    M  .= min.(β .* (S .- ELA), c)
    compute_D!(D, H, S, dSdx, dSdy, Snorm, a1, a2, dx, dy)
    qx .= avy(D) .* diff(S[:, 2:end-1], dims=1) ./ dx
    qy .= avx(D) .* diff(S[2:end-1, :], dims=2) ./ dy
    H[2:end-1, 2:end-1] .= max.(H[2:end-1, 2:end-1] .+ dt .* (diff(qx, dims=1) + diff(qy, dims=2) + M[2:end-1, 2:end-1]), 0.0)
    if mod(it, nout) == 0
        # error checking
        err = maximum(abs.(H .- H0))
        @printf("it = %d, err = %1.3e \n", it, err)
        (err < ϵ) && break
    end
end

### Post-processing

In [ ]:
# Visualise bedrock and ice elevation using GLMakie.
S_v = fill(NaN, size(S))
S_v .= S; S_v[H .<= 0.01] .= NaN
fig = Figure(; size=(1000, 450), fontsize=20)
axs = Axis3(fig[1, 1][1, 1]; xlabel="x [km]", ylabel="y [km]", zlabel="elevation [m]", zlabeloffset = 70, aspect=(4, 4, 1), azimuth=pi / 4)
plt = (p1=surface!(axs, xc ./ 1e3, yc ./ 1e3, B, colormap=Reverse(:cork)),
       p2=surface!(axs, xc ./ 1e3, yc ./ 1e3, S_v, colormap=:davos),
)
subgrid = GridLayout(fig[1, 1][1, 2], tellheight=false)
Label(subgrid[1, 1], "H ice [m]")
Colorbar(subgrid[2, 1], plt.p2; halign=:center)
resize_to_layout!(fig)
display(fig)